## 前期工作
1.写一个股票爬虫程序，实现输入任何一个股票代码和任意时间段会输出以下数据：以股票名称命名的csv文件，包含字段（日期，成交量，开盘价，最高价，最低价，收盘价）；<br>
2.基于5日K线与10日K线建立一个股票策略分析模型；

1.爬虫部分

In [121]:
import requests
import json
import csv
import datetime

class stock_spider():
    def __init__(self,stock_name,start_time,end_time):    #初始化数据，确定url中的变量
        self.stock_name = stock_name
        self.start_time = start_time
        self.end_time = end_time
        self.url = 'https://stock.xueqiu.com/v5/stock/chart/kline.json?symbol={}&begin={}&period=day&type=before&count=-{}&indicator=kline,pe,pb,ps,pcf,market_capital,agt,ggt,balance'
        #url有三个可变参数：symbol=股票代码，begin=当前发起请求的毫秒时间戳，count=获取的天数（倒序显示用-号）
        #雪球反爬比较严格，两个网页的headers必须齐全且不相同
        self.headers = {'Accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
                        'Accept-Encoding':'gzip, deflate, br',
                        'Accept-Language':'zh-CN,zh;q=0.9',
                        'Connection':'keep-alive',
                        'Cookie':'xq_a_token=87993a504d5d350e6271c337ad8e9ec8809acb79; xq_a_token.sig=71HQ_PXQYeTyQvRDRGXoyAI8Cdg; xq_r_token=2b9912fb63f07c0f11e94985018ad64e78cca498; xq_r_token.sig=QUTS2bLrXGdbA80soO-wu-fOBgY; xqat=87993a504d5d350e6271c337ad8e9ec8809acb79; xqat.sig=vnYcM61YfHCRxT0pcCqabBkxLzM; u=171573193122598; Hm_lvt_1db88642e346389874251b5a1eded6e3=1573193123; device_id=6cc4ba1894c756a9f7cca7bdef53a60c; s=cv126gt998; __guid=125150408.2502436700707036700.1573193910048.1594; Hm_lpvt_1db88642e346389874251b5a1eded6e3=1573198479; monitor_count=8',
                        'Host':'stock.xueqiu.com',
                        'Upgrade-Insecure-Requests':'1',
                        'User-Agent':'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'}
    
    
    def get_stock_code(self):   #定义函数，用于获取股票代码，以便传进self.url
        headers = {'Accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
                    'Accept-Encoding':'gzip, deflate, br',
                    'Accept-Language':'zh-CN,zh;q=0.9',
                    'Cache-Control':'max-age=0',
                    'Connection':'keep-alive',
                    'Cookie':'aliyungf_tc=AQAAAIMJ8mnCwQQAafAmGwEzuIPYEUss; acw_tc=2760826e15731931893154187e8ef5c58b388889e73c99ea661073ef7f91dd; xq_a_token=87993a504d5d350e6271c337ad8e9ec8809acb79; xq_a_token.sig=71HQ_PXQYeTyQvRDRGXoyAI8Cdg; xq_r_token=2b9912fb63f07c0f11e94985018ad64e78cca498; xq_r_token.sig=QUTS2bLrXGdbA80soO-wu-fOBgY; xqat=87993a504d5d350e6271c337ad8e9ec8809acb79; xqat.sig=vnYcM61YfHCRxT0pcCqabBkxLzM; __guid=125243267.1544968029926742000.1573193120924.3286; u=171573193122598; Hm_lvt_1db88642e346389874251b5a1eded6e3=1573193123; device_id=6cc4ba1894c756a9f7cca7bdef53a60c; s=cv126gt998; __utma=1.952561885.1573193127.1573193127.1573193127.1; __utmc=1; __utmz=1.1573193127.1.1.utmcsr=(direct)|utmccn=(direct)|utmcmd=(none); Hm_lpvt_1db88642e346389874251b5a1eded6e3=1573202257; monitor_count=24',
                    'Host':'xueqiu.com',
                    'Upgrade-Insecure-Requests':'1',
                    'User-Agent':'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'}
        url = 'https://xueqiu.com/stock/search.json?code={}&size=3&page=1'.format(self.stock_name)
        response = requests.get(url,headers=headers)
        if response.status_code == 200:
            stock_code_dic = json.loads(response.text)
            stock_code = stock_code_dic['stocks'][0]['code']
            return stock_code
        else:
            print('股票代码获取失败')
    
    
    def spider_run(self):    #定义函数，用于爬取股票数据
        begin = int(time.time())*1000
        #结束日期减去开始日期得出天数
        count = (datetime.datetime.strptime(self.end_time,'%Y-%m-%d')-datetime.datetime.strptime(self.start_time,'%Y-%m-%d')).days
        response = requests.get(self.url.format(self.get_stock_code(),begin,count),headers=self.headers)
        if response.status_code == 200:
            return json.loads(response.text)
        else:
            print('股票数据获取失败')
        
    
    def analysis(self,text):   #定义函数，用于分析并格式化获取的数据
        stock_data_dict = {}
        stock_data_list = []
        for stock_data in text['data']['item']: 
            timestamp = datetime.datetime.fromtimestamp(stock_data[0]/1000)   #将毫秒单位的时间戳转化为秒单位
            if timestamp >= datetime.datetime.strptime(self.start_time,'%Y-%m-%d'):    #只保存在开始日期和结束日期范围内的数据
                #获取对应位置数据，这里也可以不用存dict，直接遍历原始json数据，每次遍历保存至csv
                stock_data_dict['日期'] = datetime.datetime.strftime(timestamp,'%Y-%m-%d')
                stock_data_dict['成交量'] = stock_data[1]
                stock_data_dict['开盘价'] = stock_data[2]
                stock_data_dict['最高价'] = stock_data[3]
                stock_data_dict['最低价'] = stock_data[4]
                stock_data_dict['收盘价'] = stock_data[5]
                stock_data_list.append(stock_data_dict.copy())    #把格式化的每一条数据追加到列表
        return stock_data_list
        
    
    def save_data(self,list_data):
        with open(r'C:\Users\Administrator\Desktop\{}.csv'.format(self.stock_name),'w',newline='') as files:
            writer = csv.DictWriter(files,fieldnames=['日期','成交量','开盘价','最高价','最低价','收盘价'])
            writer.writeheader()
            writer.writerows(list_data)
            
if __name__ == '__main__':
    stock_class = stock_spider('格力电器','2017-1-1','2019-11-8')
    text = stock_class.spider_run()
    if text:
        list_data = stock_class.analysis(text)
        stock_class.save_data(list_data)
        print('数据已保存至桌面')
    else:
        print('没有获取到数据，请检查代码')


数据已保存至桌面


2.建模分析部分

策略一：5日K线（5日收盘价均值）与10日K线（10日收盘价均值）走势判断买卖点

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv(r'C:\Users\Administrator\Desktop\格力电器.csv',index_col = '日期',usecols = [0,5],encoding='gbk')
print(df.head())
print(df.describe())

                收盘价
日期                 
2017-01-03  22.9419
2017-01-04  23.0151
2017-01-05  23.1250
2017-01-06  22.8869
2017-01-09  22.6489
              收盘价
count  688.000000
mean    42.007147
std      9.446001
min     21.779200
25%     36.352100
50%     41.726100
75%     49.873300
max     64.450000


In [19]:
df.index = pd.to_datetime(df.index)  #将索引转换为日期格式
ma5 = df.rolling(5).mean().dropna()    #计算5日均线值,并去掉空值
ma10 = df.rolling(10).mean().dropna()    #计算10日均线值，去掉空值

In [20]:
x = len(ma5.index) - len(ma10.index)    #计算出日期相差的部分
ma5.reset_index(inplace=True)     #重新设置index，并删除相差天数
ma5.drop([i for i in range(x)],axis=0,inplace=True)
ma5.set_index('日期',drop=True,inplace=True)     #再重新把日期设置成index
ma5

,收盘价
日期,
2017-01-16,22.31384
2017-01-17,22.17470
2017-01-18,22.22596
2017-01-19,22.33032
2017-01-20,22.55188
2017-01-23,22.82470
2017-01-24,23.03526
2017-01-25,23.13414
2017-01-26,23.29892


In [25]:
'''分析买卖点：
买点：5日均线与10日均线交点后5日均线高于10日均线的第一个日期；
卖点：5日均线与10日均线交点后5日均线低于10日均线的第一个日期'''
buy_point_mod = (ma5 - ma10) > 0 
buy_point_mod

,收盘价
日期,
2017-01-16,False
2017-01-17,False
2017-01-18,False
2017-01-19,False
2017-01-20,True
2017-01-23,True
2017-01-24,True
2017-01-25,True
2017-01-26,True


In [36]:
def get_point(d):     #定义判断买卖点函数，用移动平均函数比较相邻两天的布尔值
    if d[0] == False and d[1] == True:
        return True
    elif d[0] == True and d[1] == False:
        return False
    else:
        return np.NaN
buy_point = buy_point_mod.rolling(2).apply(get_point,raw=False)
buy_point[buy_point['收盘价'] == 1.0 ] = '买入'     #格式调整
buy_point[buy_point['收盘价'] == 0.0 ] = '卖出'
buy_point.fillna('不操作',inplace=True)
buy_point.rename(columns={'收盘价':'买卖点'},inplace=True)
buy_point

,买卖点
日期,
2017-01-16,不操作
2017-01-17,不操作
2017-01-18,不操作
2017-01-19,不操作
2017-01-20,买入
2017-01-23,不操作
2017-01-24,不操作
2017-01-25,不操作
2017-01-26,不操作


In [45]:
buy_point_close = df[(df.index >= min(buy_point.index))&(df.index <= max(buy_point.index))]['收盘价']   #提取原始数据收盘价追加到买卖点中
buy_point['收盘价'] = buy_point_close
buy_point

,买卖点,收盘价
日期,,
2017-01-16,不操作,21.7792
2017-01-17,不操作,22.1271
2017-01-18,不操作,22.8503
2017-01-19,不操作,22.7771
2017-01-20,买入,23.2257
2017-01-23,不操作,23.1433
2017-01-24,不操作,23.1799
2017-01-25,不操作,23.3447
2017-01-26,不操作,23.6010


In [72]:
#计算盈利值：<br>
#1.这里只考虑全仓买卖，不考虑部分买卖，因此买卖次数关系应该是：相等或买入次数比卖出次数多一次
#2.这里也暂不考虑手续费等交易费用
print(buy_point[buy_point['买卖点'] == '买入'].count(),buy_point[buy_point['买卖点'] == '卖出'].count())   #买卖交易次数比较
profit = buy_point[buy_point['买卖点'] == '卖出']['收盘价'].values - np.delete(buy_point[buy_point['买卖点'] == '买入']['收盘价'].values,-1)
df_profit = pd.DataFrame(profit,columns=['盈亏'])
df_profit.describe()

买卖点    39
收盘价    39
dtype: int64 买卖点    38
收盘价    38
dtype: int64


,盈亏
count,38.000000
mean,0.350158
std,2.985362
min,-4.784300
25%,-1.130900
50%,-0.525000
75%,1.143325
max,8.556900


In [73]:
df_profit.sum()

盈亏    13.306
dtype: float64

## 结论：
以格力电器将近三年的每日股票数据进行分析，分析策略为5日均线和10日均线趋势走势判断买卖点，总交易38次，总收益13.3元每股，平均收益0.35元每股，当然手续费没计算在内
<br>
<br>
<br>
多嘴一句：以上分析仅供参考，千万一定别拿money去尝试，投资有风险......